In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
routes = pd.read_csv('routes.txt')
shapes = pd.read_csv('shapes.txt')
stop_times = pd.read_csv('stop_times.txt')
stops = pd.read_csv('stops.txt')
trips = pd.read_csv('trips.txt')
trips.head()

,route_id,service_id,trip_id,shape_id
0,0,1,0_6_0,0.0
1,0,1,0_6_10,0.0
2,0,1,0_6_20,0.0
3,0,1,0_6_30,0.0
4,0,1,0_6_40,0.0


In [3]:
routes.shape, shapes.shape, stop_times.shape, stops.shape, trips.shape

((1545, 5), (242244, 4), (2927174, 5), (4161, 4), (75757, 4))

In [4]:
trips.drop('service_id', axis=1, inplace=True, errors='ignore')
trips.drop('shape_id', axis=1, inplace=True, errors='ignore')
routes.drop('route_short_name', axis=1, inplace=True, errors='ignore')

In [5]:
routes[routes.route_id == 1212]

,route_long_name,route_type,route_id,agency_id
1211,124UP_DTC,3,1212,DTC


In [6]:
trips[trips.trip_id == '0_6_0']

,route_id,trip_id
0,0,0_6_0


In [14]:
trips_route = trips.set_index('route_id').join(routes.set_index('route_id'), on='route_id', how='left')
trips_route.head()

,trip_id,route_long_name,route_type,agency_id
route_id,,,,
0,0_6_0,108DOWN,3,DIMTS
0,0_6_10,108DOWN,3,DIMTS
0,0_6_20,108DOWN,3,DIMTS
0,0_6_30,108DOWN,3,DIMTS
0,0_6_40,108DOWN,3,DIMTS


In [32]:
trips_route_stopTimes = trips_route.reset_index().set_index('trip_id').join(stop_times.drop_duplicates(subset=['trip_id'], keep='first').set_index('trip_id'), on='trip_id', how='left')
trips_route_stopTimes.head()

,route_id,route_long_name,route_type,agency_id,arrival_time,departure_time,stop_id,stop_sequence
trip_id,,,,,,,,
0_6_0,0,108DOWN,3,DIMTS,06:00:00,06:00:00,2101,0
0_6_10,0,108DOWN,3,DIMTS,06:10:00,06:10:00,2101,0
0_6_20,0,108DOWN,3,DIMTS,06:20:00,06:20:00,2101,0
0_6_30,0,108DOWN,3,DIMTS,06:30:00,06:30:00,2101,0
0_6_40,0,108DOWN,3,DIMTS,06:40:00,06:40:00,2101,0


In [33]:
trips_route_stopTimes_stops = trips_route_stopTimes.reset_index().set_index('stop_id').join(stops.set_index('stop_id'), on='stop_id', how='left')
trips_route_stopTimes_stops.head()

,trip_id,route_id,route_long_name,route_type,agency_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon
stop_id,,,,,,,,,,,
2101,0_6_0,0,108DOWN,3,DIMTS,06:00:00,06:00:00,0,Hari Nagar Clock Tower,28.6252,77.1108
2101,0_6_10,0,108DOWN,3,DIMTS,06:10:00,06:10:00,0,Hari Nagar Clock Tower,28.6252,77.1108
2101,0_6_20,0,108DOWN,3,DIMTS,06:20:00,06:20:00,0,Hari Nagar Clock Tower,28.6252,77.1108
2101,0_6_30,0,108DOWN,3,DIMTS,06:30:00,06:30:00,0,Hari Nagar Clock Tower,28.6252,77.1108
2101,0_6_40,0,108DOWN,3,DIMTS,06:40:00,06:40:00,0,Hari Nagar Clock Tower,28.6252,77.1108


In [34]:
df = trips_route_stopTimes_stops
date_type = ['arrival_time', 'departure_time']
categorical = ['route_long_name', 'route_type', 'agency_id', 'stop_name']

In [35]:
# df = pd.get_dummies(df, prefix=categorical, columns=categorical)
le = LabelEncoder()
for col in categorical:
    df[col] = le.fit_transform(df[col])

In [40]:
df.arrival_time = pd.to_timedelta(df.arrival_time).astype(int) / 10**9
df.departure_time = pd.to_timedelta(df.departure_time).astype(int) / 10**9
df = df.reset_index()
df.shape

(75757, 14)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('route_id', axis=1), df['route_id'], test_size=0.2) 

In [44]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/Users/macbookpro/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [45]:
lr.score(X_test, y_test)

0.001187961985216473